In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("../files/chapter_three.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(llm=llm,
                                  max_token_limit=200,
                                  return_messages=True,
                                  memory_key="chat_history")



def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]




prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = {
    "context": retriever,
    "question": RunnablePassthrough(),
    "chat_history": load_memory,
}  | prompt | llm

chain
def invoke_chain(question):
    result = chain.invoke(question).content
    memory.save_context(
        {'input': question},
        {'output': result},
    )
    print(result)
    



In [2]:
invoke_chain("Is Aaronson guilty?")

I don't know.


In [3]:
invoke_chain("What message did he write in the table?")

He traced with his finger in the dust on the table: 2+2=5.


In [4]:
invoke_chain("Who is Julia?")

Julia is a character in the novel "1984" by George Orwell. She is Winston Smith's love interest and represents rebellion against the oppressive regime of the Party. Julia is known for her passionate nature and her desire for personal freedom and intimacy in a society that suppresses individualism and emotional connections.


In [5]:
invoke_chain("what was my all questions and answers?")

1. **Question:** Is Aaronson guilty?  
   **Answer:** I don't know.

2. **Question:** What message did he write in the table?  
   **Answer:** He traced with his finger in the dust on the table: 2+2=5.

3. **Question:** Who is Julia?  
   **Answer:** Julia is a character in the novel "1984" by George Orwell. She is Winston Smith's love interest and represents rebellion against the oppressive regime of the Party. Julia is known for her passionate nature and her desire for personal freedom and intimacy in a society that suppresses individualism and emotional connections.
